# Data Augmentation

In this notebook we will explore how to perform data augment data as shown in lecture 5. Its essential when using deep networks to get the most out of your data. Fortunately, performing keras makes augmenting data with its ImageDataGenerator

(This first line is completely unnecessary - I changed my mind on what this notebook would contain. But, if you need to add new libraries in Google Colab, this is how you do it. I thought it might be a useful thing for you to know about so I've left it here.)

In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

     |████████████████████████████████| 3.8MB 5.9MB/s 


Lets start with our imports, as usual.

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.losses import categorical_crossentropy
from keras import backend as K

This time we will load the fashion mnist data set.

In [ ]:
batch_size = 32
epochs = 100
num_classes=10

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train.shape

img_rows = 28
img_cols = 28

4423680/4422102 [==============================] - 0s 0us/step


Hopefully a lot of this code looks familiar to you by now

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


In [ ]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Lets also copy the model from the last notebook because it seemed to work quite well. (Note: how much nicer would it look if we just had a model.py file to store this code and then simply called deep_model from there. You can try this as an exercise if you wish)

In [ ]:
deep_model = Sequential()
deep_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1), padding='same')) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu', padding='same', strides=2)) 
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3)) 
deep_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
deep_model.add(BatchNormalization())
deep_model.add(Dropout(0.3))
deep_model.add(Flatten()) # This line is to convert from matrices to vectors
deep_model.add(Dense(128, activation='relu'))
deep_model.add(Dropout(0.3))
deep_model.add(Dense(10, activation='softmax')) 
deep_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9

Finally, lets compile the model again

In [ ]:
deep_model.compile(loss=categorical_crossentropy,
              optimizer='adam', #Note that I have used adam here instead of sgd. It is a little bit more advanced.
              metrics=['accuracy'])

Now we have the data loaded and the model compiled, let's explore how we can do data augmentation

---



In [ ]:
from keras.preprocessing.image import ImageDataGenerator    
    
datagen = ImageDataGenerator(
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=30,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random zoom
    zoom_range=0.5,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # We will usually flip images horizontally rather than vertically as these flips are more realistic in real life
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(x_train)
# datagen.flow() will apply your augmentations specified above to each image in the batch.
deep_model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=1)

Epoch 1/150
1875/1875 [==============================] - 57s 14ms/step - loss: 1.7089 - accuracy: 0.3922 - val_loss: 0.7645 - val_accuracy: 0.6963
Epoch 2/150
1875/1875 [==============================] - 25s 13ms/step - loss: 0.9932 - accuracy: 0.6324 - val_loss: 0.6845 - val_accuracy: 0.7341
Epoch 3/150
1875/1875 [==============================] - 25s 14ms/step - loss: 0.8679 - accuracy: 0.6807 - val_loss: 0.5911 - val_accuracy: 0.7846
Epoch 4/150
1875/1875 [==============================] - 25s 13ms/step - loss: 0.7879 - accuracy: 0.7137 - val_loss: 0.6122 - val_accuracy: 0.7670
Epoch 5/150
1875/1875 [==============================] - 25s 13ms/step - loss: 0.7256 - accuracy: 0.7339 - val_loss: 0.6156 - val_accuracy: 0.7707
Epoch 6/150
1875/1875 [==============================] - 25s 13ms/step - loss: 0.6743 - accuracy: 0.7564 - val_loss: 0.5700 - val_accuracy: 0.7899
Epoch 7/150
1875/1875 [==============================] - 25s 13ms/step - loss: 0.6511 - accuracy: 0.7667 - val_loss: 0

To learn more about the data augmentations available, see [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

Note that this model is far from optimised, we don't get close to 100% training accuracy so you could definitely improve on this. But the data augmentation has led to us not overfitting.
